In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import altair as alt
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.ardl import ARDL
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Data

In [ ]:
export_data = pd.read_csv('export2003-2024.csv', sep=';')
print('in USD')
export_data

in USD


,reporter,com_label,commodity,year,export,import
0,Indonesia,0,[01] Animals; live,2003,2.511837e+07,82754595
1,Indonesia,0,[02] Meat and edible meat offal,2003,2.177500e+07,44884944
2,Indonesia,0,"[03] Fish and crustaceans, molluscs and other ...",2003,1.437417e+09,30132383
3,Indonesia,0,[04] Dairy produce; birds' eggs; natural honey...,2003,1.082529e+08,277225879
4,Indonesia,0,[05] Animal originated products; not elsewhere...,2003,4.109344e+06,6504851
...,...,...,...,...,...,...
2129,Indonesia,9,"[94] Furniture; bedding, mattresses, mattress ...",2024,2.427474e+09,1770940291
2130,Indonesia,9,"[95] Toys, games and sports requisites; parts ...",2024,7.664741e+08,585605581
2131,Indonesia,9,[96] Miscellaneous manufactured articles,2024,3.481782e+08,753185077
2132,Indonesia,9,[97] Works of art; collectors' pieces and anti...,2024,6.972042e+06,28392968


In [ ]:
price_index_data = pd.read_csv('whoprice2003-2024.csv', sep=';')
price_index_data.tail()

,year,who_com_prices,energy_index,non_energy_index,precious_metal_index
17,2020,63.07,52.68,84.12,133.54
18,2021,100.90,95.38,112.09,140.17
19,2022,142.52,152.57,122.14,136.80
20,2023,108.04,106.95,110.24,147.34
21,2024,105.14,101.52,112.47,180.17


In [ ]:
gdp_data = pd.read_csv('gdp2003-2024.csv', sep=';')
print('in IDR')
gdp_data.tail()

in IDR


,year,export,import,gdp
17,2020,2676513530000000,2415495970000000,15443353200000000
18,2021,3635835410000000,3189626450000000,16976751400000000
19,2022,4799815440000000,4106085660000000,19588089900000000
20,2023,4543377780000000,4088446950000000,20892376700000000
21,2024,4911155650000000,4513987410000000,22138964000000000


# Export-Import Analysis

In [ ]:
export_data['trade_balance'] = export_data['export'] - export_data['import']
# export_data['export_p'] = export_data['export'] / export_data['export'].sum() * 100
# export_data['import_p'] = export_data['import'] / export_data['export'].sum() * 100
export_data['com_label_name'] = export_data['com_label'].map({
    0: 'Food and animals',
    1: 'Beverages and cigarettes',
    2: 'Raw materials, edible, except fuels',
    3: 'Minerals, fuels and other',
    4: 'Animal & vegetable oils and fats',
    5: 'Chemicals and other products',
    6: 'Basic manufactures',
    7: 'Machinery, transport, equipment',
    8: 'Other manufactured goods',
    9: 'Other unclassifiable goods'
})
export_data.tail()

,reporter,com_label,commodity,year,export,import,trade_balance,com_label_name
2129,Indonesia,9,"[94] Furniture; bedding, mattresses, mattress ...",2024,2.427474e+09,1770940291,6.565340e+08,Other unclassifiable goods
2130,Indonesia,9,"[95] Toys, games and sports requisites; parts ...",2024,7.664741e+08,585605581,1.808685e+08,Other unclassifiable goods
2131,Indonesia,9,[96] Miscellaneous manufactured articles,2024,3.481782e+08,753185077,-4.050069e+08,Other unclassifiable goods
2132,Indonesia,9,[97] Works of art; collectors' pieces and anti...,2024,6.972042e+06,28392968,-2.142093e+07,Other unclassifiable goods
2133,Indonesia,9,[99] Other products,2024,1.974912e+07,1168642295,-1.148893e+09,Other unclassifiable goods


## Commodities Traded

This is summation of export and import data for HS-1 Commodity for 2003 - 2024

In [ ]:
overall_export_data = export_data.groupby('com_label_name')[['export', 'import']].sum().sort_values('import', ascending=False)
overall_export_data['export_share'] = overall_export_data['export'] / overall_export_data['export'].sum()
overall_export_data['import_share'] = overall_export_data['import'] / overall_export_data['import'].sum()
overall_export_data = overall_export_data.sort_values(by='export_share', ascending=False)
overall_export_data

,export,import,export_share,import_share
com_label_name,,,,
"Raw materials, edible, except fuels",1.159074e+12,893961893139,0.320289,0.279888
Other manufactured goods,5.017360e+11,1012788573604,0.138646,0.317091
Beverages and cigarettes,4.783941e+11,166091345919,0.132196,0.052001
"Machinery, transport, equipment",3.728929e+11,332263771060,0.103042,0.104027
Animal & vegetable oils and fats,3.652671e+11,120239587328,0.100935,0.037645
Basic manufactures,2.609218e+11,60965209120,0.072101,0.019087
"Minerals, fuels and other",1.830889e+11,303220197182,0.050593,0.094934
Food and animals,1.171328e+11,84882481572,0.032368,0.026576
Other unclassifiable goods,9.236401e+10,108226217796,0.025523,0.033884


In [ ]:
fig = px.treemap(export_data, path=['com_label_name', 'commodity'], values="export", color='com_label_name', title='Overall Export 2003-2024 Treemap by Commodities')
fig.update_layout(margin = dict(t=50, l=0, r=0, b=0))
fig.show()

In [ ]:
export_data[export_data['com_label_name']=='Raw materials, edible, except fuels'].groupby(['year', 'com_label_name'])[['export']].sum()

,,export
year,com_label_name,
2003,"Raw materials, edible, except fuels",1.982590e+10
2004,"Raw materials, edible, except fuels",2.310076e+10
2005,"Raw materials, edible, except fuels",3.012682e+10
2006,"Raw materials, edible, except fuels",3.612965e+10
2007,"Raw materials, edible, except fuels",3.205688e+10
2008,"Raw materials, edible, except fuels",4.847431e+10
2009,"Raw materials, edible, except fuels",4.238881e+10
2010,"Raw materials, edible, except fuels",5.989589e+10
2011,"Raw materials, edible, except fuels",8.300821e+10


In [ ]:
fig = px.treemap(export_data, path=['com_label_name', 'commodity'], values="import", color='com_label_name', title='Overall Import 2003-2024 Treemap by Commodities')
fig.update_layout(margin = dict(t=50, l=0, r=0, b=0))
fig.show()

In [ ]:
export_data[export_data['com_label_name']=='Other manufactured goods'].groupby(['year', 'com_label_name'])[['import']].sum()

,,import
year,com_label_name,
2003,Other manufactured goods,8758537129
2004,Other manufactured goods,12451626729
2005,Other manufactured goods,15622205511
2006,Other manufactured goods,15825144084
2007,Other manufactured goods,15698083274
2008,Other manufactured goods,43652582114
2009,Other manufactured goods,36544316349
2010,Other manufactured goods,47896166933
2011,Other manufactured goods,57285388436


In [ ]:
export_grouped = export_data.groupby(['year', 'com_label_name'], as_index=False)['export'].sum()

fig = px.line(export_grouped,
              x='year',
              y='export',
              color='com_label_name',
              markers=True,  # Add markers for clarity
              title='Yearly Export Value by HS-1 Digit',
              labels={'export': 'Total Export Value', 'year': 'Year', 'com_label_name': 'Commodity'},
              template='plotly_white')  # Clean background style
fig.update_xaxes(tickmode='linear')
fig.update_layout(width=1100, height=400)
fig.show()

In [ ]:
import_grouped = export_data.groupby(['year', 'com_label_name'], as_index=False)['import'].sum()

fig = px.line(import_grouped,
              x='year',
              y='import',
              color='com_label_name',
              markers=True,  # Add markers for clarity
              title='Yearly Import Value by HS-1 Digit',
              labels={'import': 'Total Import Value', 'year': 'Year', 'com_label_name': 'Commodity'},
              template='plotly_white')  # Clean background style
fig.update_xaxes(tickmode='linear')
fig.update_layout(width=1100, height=400)
fig.show()

In [ ]:
yearly_export_data = export_data.groupby('year')[['export', 'import', 'trade_balance']].sum()
yearly_export_data = yearly_export_data.reset_index()
yearly_export_data['export_growth'] = yearly_export_data['export'].pct_change() * 100
yearly_export_data['import_growth'] = yearly_export_data['import'].pct_change() * 100
yearly_export_data

,year,export,import,trade_balance,export_growth,import_growth
0,2003,6.105825e+10,32550690611,2.850756e+10,NaN,NaN
1,2004,7.158461e+10,46524531358,2.506008e+10,17.239869,42.929475
2,2005,8.565995e+10,57700882616,2.795907e+10,19.662528,24.022491
3,2006,1.007986e+11,61065465536,3.973316e+10,17.672986,5.831077
4,2007,1.141009e+11,74473430118,3.962746e+10,13.196873,21.956706
5,2008,1.370204e+11,129197306224,7.823118e+09,20.087077,73.481074
6,2009,1.165100e+11,96829162576,1.968083e+10,-14.968887,-25.053265
7,2010,1.577791e+11,135663284048,2.211582e+10,35.421092,40.105812
8,2011,2.034966e+11,177435555736,2.606107e+10,28.975651,30.791140
9,2012,1.900318e+11,191689471527,-1.657626e+09,-6.616710,8.033292


In [ ]:
print('in Trillion USD')
(yearly_export_data[['export', 'import', 'trade_balance']]/1_000_000_000_000).agg(['sum', 'mean'])

in Trillion USD


,export,import,trade_balance
sum,3.618833,3.194002,0.424831
mean,0.164492,0.145182,0.019311


### HS-2 Share

In [ ]:
commodity_export_per_year = pd.merge(yearly_export_data.rename(columns={'export': 'total_export', 'import': 'total_import'}), export_data, on='year')
commodity_export_per_year['export_share'] = commodity_export_per_year['export']/commodity_export_per_year['total_export']
commodity_export_per_year['import_share'] = commodity_export_per_year['import']/commodity_export_per_year['total_import']
commodity_export_per_year = commodity_export_per_year.drop(['trade_balance_x', 'trade_balance_y', 'export_growth', 'import_growth', 'reporter', 'total_export', 'total_import'], axis=1)

**Export**

In [ ]:
commodity_export_share_per_year = commodity_export_per_year.sort_values(by=['year', 'export_share'], ascending=[True, False]).drop(['import_share', 'import'], axis=1)
commodity_export_share_per_year

,year,com_label,commodity,export,com_label_name,export_share
26,2003,2,"[27] Mineral fuels, mineral oils and products ...",1.571298e+10,"Raw materials, edible, except fuels",2.573441e-01
83,2003,8,[85] Electrical machinery and equipment and pa...,6.118953e+09,Other manufactured goods,1.002150e-01
43,2003,4,[44] Wood and articles of wood; wood charcoal,3.180501e+09,Animal & vegetable oils and fats,5.208961e-02
14,2003,1,[15] Animal or vegetable fats and oils and the...,3.003362e+09,Beverages and cigarettes,4.918847e-02
82,2003,8,"[84] Nuclear reactors, boilers, machinery and ...",2.787126e+09,Other manufactured goods,4.564701e-02
...,...,...,...,...,...,...
2102,2024,6,"[66] Umbrellas, sun umbrellas, walking-sticks,...",1.988018e+06,Basic manufactures,7.510289e-06
2079,2024,4,[43] Furskins and artificial fur; manufactures...,7.102266e+05,Animal & vegetable oils and fats,2.683078e-06
2087,2024,5,"[51] Wool, fine or coarse animal hair; horseha...",4.672230e+05,Chemicals and other products,1.765064e-06
2081,2024,4,[45] Cork and articles of cork,2.035437e+05,Animal & vegetable oils and fats,7.689426e-07


In [ ]:
commodity_export_share_pivot_table = commodity_export_share_per_year.pivot_table(index='commodity', columns='year', values='export_share')
commodity_export_share_pivot_table['avg_share'] = commodity_export_share_pivot_table.mean(axis=1)
commodity_export_share_pivot_table = commodity_export_share_pivot_table.sort_values(by='avg_share', ascending=False)
commodity_export_share_pivot_table.to_csv('awikwik.csv', sep='|')
print("\nSorted and Formatted Pivot Table (Export Share as Percentages, Max Highlighted)")
display(commodity_export_share_pivot_table.style.format("{:.2%}").highlight_max(axis=0, color='#ffffb3'))


Sorted and Formatted Pivot Table (Export Share as Percentages, Max Highlighted)


year,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,avg_share
commodity,,,,,,,,,,,,,,,,,,,,,,,
"[27] Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",25.73%,25.93%,27.69%,27.40%,19.81%,29.03%,28.28%,29.64%,33.87%,33.35%,31.45%,29.02%,23.01%,19.21%,21.85%,23.21%,20.34%,15.63%,19.46%,24.32%,22.98%,20.97%,25.10%
[15] Animal or vegetable fats and oils and their cleavage products; prepared animal fats; animal or vegetable waxes,4.92%,6.18%,5.78%,6.02%,8.96%,11.40%,10.49%,10.34%,10.64%,11.21%,10.53%,11.97%,12.42%,12.56%,13.60%,11.30%,10.52%,12.69%,14.21%,12.04%,10.99%,10.13%,10.41%
"[85] Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",10.02%,9.18%,8.56%,7.23%,6.64%,6.01%,6.94%,6.57%,5.48%,5.66%,5.72%,5.54%,5.70%,5.62%,5.04%,4.92%,5.37%,5.66%,5.08%,4.98%,5.54%,5.69%,6.23%
[40] Rubber and articles thereof,3.48%,4.19%,4.18%,5.49%,5.48%,5.57%,4.22%,5.94%,7.05%,5.51%,5.15%,4.03%,3.93%,3.90%,4.58%,3.54%,3.59%,3.44%,3.07%,2.19%,1.97%,2.08%,4.21%
"[84] Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof",4.56%,5.38%,5.32%,4.33%,4.17%,3.85%,4.13%,3.16%,2.83%,3.21%,3.27%,3.39%,3.47%,4.06%,3.48%,3.26%,3.31%,3.19%,2.72%,2.38%,2.50%,2.62%,3.57%
"[26] Ores, slag and ash",3.17%,2.70%,4.09%,4.95%,4.47%,3.14%,4.98%,5.16%,3.61%,2.67%,3.58%,1.09%,2.25%,2.46%,2.23%,2.92%,1.86%,1.98%,2.74%,3.53%,3.37%,3.11%,3.19%
[72] Iron and steel,0.66%,0.96%,0.83%,1.25%,0.98%,1.23%,0.73%,0.70%,0.66%,0.46%,0.36%,0.65%,0.80%,1.26%,1.98%,3.20%,4.41%,6.66%,9.03%,9.52%,10.32%,9.75%,3.02%
"[87] Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",1.03%,1.31%,1.52%,1.65%,1.85%,2.06%,1.56%,1.84%,1.64%,2.56%,2.50%,2.96%,3.61%,4.04%,4.05%,4.20%,4.88%,4.05%,3.73%,3.76%,4.31%,4.16%,2.87%
[62] Apparel and clothing accessories; not knitted or crocheted,4.28%,3.93%,3.59%,3.35%,2.90%,2.48%,2.69%,2.29%,2.04%,1.97%,2.14%,2.23%,2.65%,2.67%,2.45%,2.50%,2.67%,2.23%,1.78%,1.67%,1.63%,1.61%,2.53%


**Import**

In [ ]:
commodity_import_share_per_year = commodity_export_per_year.sort_values(by=['year', 'import_share'], ascending=[True, False]).drop(['export_share', 'export'], axis=1)
commodity_import_share_per_year

,year,com_label,commodity,import,com_label_name,import_share
26,2003,2,"[27] Mineral fuels, mineral oils and products ...",7667923941,"Raw materials, edible, except fuels",0.235569
82,2003,8,"[84] Nuclear reactors, boilers, machinery and ...",4303040825,Other manufactured goods,0.132195
28,2003,2,[29] Organic chemicals,2163896816,"Raw materials, edible, except fuels",0.066478
85,2003,8,[87] Vehicles; other than railway or tramway r...,1890290768,Other manufactured goods,0.058072
83,2003,8,[85] Electrical machinery and equipment and pa...,1763077039,Other manufactured goods,0.054164
...,...,...,...,...,...,...
2093,2024,5,[57] Carpets and other textile floor coverings,8425094,Chemicals and other products,0.000036
2081,2024,4,[45] Cork and articles of cork,6054779,Animal & vegetable oils and fats,0.000026
2050,2024,1,[14] Vegetable plaiting materials; vegetable p...,3090436,Beverages and cigarettes,0.000013
2086,2024,5,[50] Silk,2363218,Chemicals and other products,0.000010


In [ ]:
commodity_import_share_pivot_table = commodity_import_share_per_year.pivot_table(index='commodity', columns='year', values='import_share')
commodity_import_share_pivot_table['avg_share'] = commodity_import_share_pivot_table.mean(axis=1)
commodity_import_share_pivot_table = commodity_import_share_pivot_table.sort_values(by='avg_share', ascending=False)
commodity_import_share_pivot_table.to_csv('awikwik2.csv', sep="|")
print("\nSorted and Formatted Pivot Table (Import Share as Percentages, Max Highlighted)")
display(commodity_import_share_pivot_table.style.format("{:.2%}").highlight_max(axis=0, color='#ffffb3'))


Sorted and Formatted Pivot Table (Import Share as Percentages, Max Highlighted)


year,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,avg_share
commodity,,,,,,,,,,,,,,,,,,,,,,,
"[27] Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",23.56%,25.37%,30.36%,31.18%,17.33%,23.75%,19.72%,20.29%,23.02%,22.31%,24.40%,24.66%,17.55%,14.19%,16.24%,16.74%,13.71%,11.15%,14.70%,18.91%,18.08%,17.40%,20.21%
"[84] Nuclear reactors, boilers, machinery and mechanical appliances; parts thereof",13.22%,13.58%,14.01%,12.14%,10.50%,15.19%,16.01%,14.76%,13.94%,14.83%,14.62%,14.50%,15.68%,15.53%,13.87%,14.41%,15.66%,15.40%,13.17%,13.30%,14.49%,14.34%,14.23%
"[85] Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles",5.42%,5.95%,5.75%,5.07%,4.68%,10.13%,10.85%,11.52%,10.28%,9.86%,9.75%,9.67%,10.88%,11.38%,11.42%,11.37%,11.71%,13.44%,11.39%,11.12%,11.62%,11.58%,9.76%
[72] Iron and steel,3.95%,5.84%,5.80%,4.69%,6.79%,6.41%,4.50%,4.70%,4.84%,5.29%,5.12%,4.69%,4.43%,4.56%,5.09%,5.43%,6.07%,4.84%,6.09%,5.87%,5.13%,4.56%,5.21%
"[87] Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof",5.81%,5.21%,5.31%,4.01%,3.02%,5.15%,4.01%,4.23%,4.28%,5.09%,4.24%,3.51%,3.74%,3.91%,4.26%,4.28%,4.18%,3.13%,3.42%,4.00%,4.60%,4.13%,4.25%
[29] Organic chemicals,6.65%,7.00%,5.62%,5.63%,4.73%,3.85%,3.94%,3.93%,3.74%,3.59%,3.76%,3.97%,4.01%,3.53%,3.76%,3.67%,3.40%,3.55%,3.72%,3.25%,2.89%,3.04%,4.15%
[39] Plastics and articles thereof,3.49%,3.45%,3.03%,3.04%,2.65%,3.03%,3.30%,3.55%,3.77%,3.65%,4.09%,4.37%,4.79%,5.16%,4.92%,4.88%,5.13%,5.05%,5.19%,4.68%,4.24%,4.53%,4.09%
[10] Cereals,3.21%,2.32%,1.53%,2.01%,4.24%,1.70%,1.56%,1.59%,2.68%,1.94%,1.94%,2.02%,2.21%,2.35%,1.86%,2.01%,1.89%,2.13%,2.08%,1.88%,2.68%,2.92%,2.22%
[73] Iron or steel articles,2.12%,1.85%,2.57%,2.07%,1.70%,2.58%,2.88%,2.54%,2.01%,2.55%,2.54%,2.41%,2.60%,2.16%,1.67%,2.06%,2.13%,1.97%,1.60%,1.68%,1.96%,1.81%,2.16%


## Year Average Growth

In [ ]:
yearly_export_data = yearly_export_data.reset_index()
yearly_export_data_melted = yearly_export_data.melt(id_vars=['year'], value_vars=['export', 'import'], var_name='Trade Type', value_name='Value')
fig = px.line(yearly_export_data_melted, x='year', y='Value', color='Trade Type', markers=True)
fig.update_layout(xaxis_title="Year", yaxis_title="Value US$", template="plotly_white")
fig.update_xaxes(tickmode='linear')
fig.show()

In [ ]:
export_avg_growth = yearly_export_data['export_growth'].mean()
import_avg_growth = yearly_export_data['import_growth'].mean()
print(f"Export yearly-averaged growth: {export_avg_growth:.2f}%")
print(f"Import yearly-averaged growth: {import_avg_growth:.2f}%")

Export yearly-averaged growth: 8.45%
Import yearly-averaged growth: 12.28%


## Compound Annual Growth Rate
![image.png](attachment:878377c8-31c1-412c-be1f-cc5eff0190e5.png)

In [ ]:
N = len(yearly_export_data) - 1

export_0 = yearly_export_data['export'].iloc[0]
export_N = yearly_export_data['export'].iloc[-1]

export_cagr = ((export_N / export_0) ** (1 / N) - 1) * 100
print(f"Export CAGR: {export_cagr:.2f}%")

import_0 = yearly_export_data['import'].iloc[0]
import_N = yearly_export_data['import'].iloc[-1]

import_cagr = ((import_N / import_0) ** (1 / N) - 1) * 100
print(f"Import CAGR: {import_cagr:.2f}%")

Export CAGR: 7.23%
Import CAGR: 9.84%


In [ ]:
merged_ex_price_data = yearly_export_data.merge(price_index_data, left_on='year', right_on='year').drop('index', axis=1)
merged_ex_price_data.head()

,year,export,import,trade_balance,export_growth,import_growth,who_com_prices,energy_index,non_energy_index,precious_metal_index
0,2003,6.105825e+10,32550690611,2.850756e+10,NaN,NaN,43.83,41.40,48.76,29.08
1,2004,7.158461e+10,46524531358,2.506008e+10,17.239869,42.929475,54.49,53.44,56.60,33.97
2,2005,8.565995e+10,57700882616,2.795907e+10,19.662528,24.022491,70.74,75.39,61.29,36.95
3,2006,1.007986e+11,61065465536,3.973316e+10,17.672986,5.831077,82.16,85.40,75.56,51.57
4,2007,1.141009e+11,74473430118,3.962746e+10,13.196873,21.956706,92.89,94.37,89.82,59.49


# Granger Causality

In [ ]:
merged_data_all = merged_ex_price_data.merge(gdp_data.drop(['export', 'import'], axis=1), on='year')
merged_data_all

,year,export,import,trade_balance,export_growth,import_growth,who_com_prices,energy_index,non_energy_index,precious_metal_index,gdp
0,2003,6.105825e+10,32550690611,2.850756e+10,NaN,NaN,43.83,41.40,48.76,29.08,2013674600000000
1,2004,7.158461e+10,46524531358,2.506008e+10,17.239869,42.929475,54.49,53.44,56.60,33.97,2295826200000000
2,2005,8.565995e+10,57700882616,2.795907e+10,19.662528,24.022491,70.74,75.39,61.29,36.95,2774281100000000
3,2006,1.007986e+11,61065465536,3.973316e+10,17.672986,5.831077,82.16,85.40,75.56,51.57,3339216800000000
4,2007,1.141009e+11,74473430118,3.962746e+10,13.196873,21.956706,92.89,94.37,89.82,59.49,3950893200000000
5,2008,1.370204e+11,129197306224,7.823118e+09,20.087077,73.481074,121.38,129.03,105.72,72.66,4948688400000000
6,2009,1.165100e+11,96829162576,1.968083e+10,-14.968887,-25.053265,80.88,79.36,83.94,78.01,5606203400000000
7,2010,1.577791e+11,135663284048,2.211582e+10,35.421092,40.105812,100.00,100.00,100.00,100.00,6864133100000000
8,2011,2.034966e+11,177435555736,2.606107e+10,28.975651,30.791140,130.07,135.22,119.56,136.27,7831726000000000
9,2012,1.900318e+11,191689471527,-1.657626e+09,-6.616710,8.033292,126.01,134.12,109.51,138.46,8615704500000000


In [ ]:
def check_stationarity(series):
    result = adfuller(series.dropna())
    print(f"ADF Statistic: {result[0]}")
    print(f"P-Value: {result[1]}")
    return result[1] < 0.05

def make_stationary(data, columns):
    for col in columns:
        if data[col].isna().any() or (data[col] == float('inf')).any():
            print(f"Column {col} has NaN or infinite values, cleaning it.")
            data[col] = data[col].fillna(method='ffill').fillna(method='bfill')

        while not check_stationarity(data[col]):
            print(f"Column {col} is not stationary. Differencing applied.")
            data[col] = data[col].diff().dropna()
            if data[col].isna().any():
                print(f"Column {col} has NaN values after differencing. Dropping NaNs.")
                data[col] = data[col].dropna()
        print(f"Column {col} is stationary now.", end="\n\n")

## ADF and Differencing

In [ ]:
# make_stationary(gdp_data, ['export', 'import', 'gdp', ])
make_stationary(merged_data_all, ['export', 'import', 'gdp', 'who_com_prices'])
merged_data_all.dropna(inplace=True)
merged_data_all.head()

ADF Statistic: 0.19289192330432417
P-Value: 0.9718600938153541
Column export is not stationary. Differencing applied.
Column export has NaN values after differencing. Dropping NaNs.
ADF Statistic: -3.964803882195709
P-Value: 0.001606446155133737
Column export is stationary now.

ADF Statistic: -1.5568733130418417
P-Value: 0.5052131171009894
Column import is not stationary. Differencing applied.
Column import has NaN values after differencing. Dropping NaNs.
ADF Statistic: -3.21674058443748
P-Value: 0.01903588908649944
Column import is stationary now.

ADF Statistic: 2.9336277206229022
P-Value: 1.0
Column gdp is not stationary. Differencing applied.
Column gdp has NaN values after differencing. Dropping NaNs.
ADF Statistic: 0.6270400807020705
P-Value: 0.9882634999948895
Column gdp is not stationary. Differencing applied.
Column gdp has NaN values after differencing. Dropping NaNs.
ADF Statistic: -6.078806098132146
P-Value: 1.1048963365714077e-07
Column gdp is stationary now.

ADF Statis

,year,export,import,trade_balance,export_growth,import_growth,who_com_prices,energy_index,non_energy_index,precious_metal_index,gdp
2,2005,1.407534e+10,1.117635e+10,2.795907e+10,19.662528,24.022491,70.74,75.39,61.29,36.95,1.963033e+14
3,2006,1.513867e+10,3.364583e+09,3.973316e+10,17.672986,5.831077,82.16,85.40,75.56,51.57,8.648080e+13
4,2007,1.330227e+10,1.340796e+10,3.962746e+10,13.196873,21.956706,92.89,94.37,89.82,59.49,4.674070e+13
5,2008,2.291953e+10,5.472388e+10,7.823118e+09,20.087077,73.481074,121.38,129.03,105.72,72.66,3.861188e+14
6,2009,-2.051043e+10,-3.236814e+10,1.968083e+10,-14.968887,-25.053265,80.88,79.36,83.94,78.01,-3.402802e+14


## Causality Test

In [ ]:
merged_data_all = merged_data_all.set_index('year')

In [ ]:
model = VAR(merged_data_all[['gdp', 'export', 'import', 'who_com_prices']])
lag_order = model.select_order(maxlags=3)
print(lag_order.summary())
optimal_lag = lag_order.aic

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0       169.1       169.3   2.665e+73       169.1
1       168.0      169.0*  9.947e+72*       168.1
2       167.6       169.4   1.024e+73       167.8
3      167.3*       169.8   3.079e+73      167.5*
-------------------------------------------------


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.



### Individual Inferential Statistics

In [ ]:
var_model = model.fit(optimal_lag)
var_model.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 28, Jun, 2025
Time:                     10:58:50
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    169.820
Nobs:                     17.0000    HQIC:                   167.525
Log likelihood:          -1466.29    FPE:                3.07863e+73
AIC:                      167.271    Det(Omega_mle):     3.17445e+72
--------------------------------------------------------------------
Results for equation gdp
                              coefficient               std. error           t-stat            prob
---------------------------------------------------------------------------------------------------
const                -831599217220.236206  1057665077661312.625000           -0.001           0.999
L1.gdp                          -1.115684                 0.385011           -2.898   

### Joint Causality

In [ ]:
granger_test = var_model.test_causality(caused='gdp', causing='export', kind='f', signif=0.1)
print(granger_test.summary())

Granger causality F-test. H_0: export does not Granger-cause gdp. Conclusion: fail to reject H_0 at 10% significance level.
Test statistic Critical value p-value         df       
-------------------------------------------------------
         1.624          2.462   0.223 (3, np.int64(16))
-------------------------------------------------------


In [ ]:
granger_test = var_model.test_causality(caused='gdp', causing='import', kind='f', signif=0.1)
print(granger_test.summary())

Granger causality F-test. H_0: import does not Granger-cause gdp. Conclusion: fail to reject H_0 at 10% significance level.
Test statistic Critical value p-value         df       
-------------------------------------------------------
         2.112          2.462   0.139 (3, np.int64(16))
-------------------------------------------------------


In [ ]:
granger_test = var_model.test_causality(caused='export', causing='who_com_prices', kind='f', signif=0.1)
print(granger_test.summary())

Granger causality F-test. H_0: who_com_prices does not Granger-cause export. Conclusion: fail to reject H_0 at 10% significance level.
Test statistic Critical value p-value         df       
-------------------------------------------------------
        0.6210          2.462   0.612 (3, np.int64(16))
-------------------------------------------------------


In [ ]:
granger_test = var_model.test_causality(caused='import', causing='who_com_prices', kind='f', signif=0.1)
print(granger_test.summary())

Granger causality F-test. H_0: who_com_prices does not Granger-cause import. Conclusion: fail to reject H_0 at 10% significance level.
Test statistic Critical value p-value         df       
-------------------------------------------------------
        0.3846          2.462   0.766 (3, np.int64(16))
-------------------------------------------------------


# Forecasting

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
errors = {}

In [ ]:
yearly_trade_data_cleaned = yearly_export_data.merge(price_index_data, on='year')[['year', 'export', 'import', 'who_com_prices']]
yearly_trade_data_cleaned['year'] = pd.to_datetime(yearly_trade_data_cleaned['year'], format='%Y')
yearly_trade_data_cleaned = yearly_trade_data_cleaned.set_index('year')
yearly_trade_data_cleaned = yearly_trade_data_cleaned.asfreq('YS')

train_size = int(len(yearly_trade_data_cleaned) * 0.8)
train, test = yearly_trade_data_cleaned[:train_size], yearly_trade_data_cleaned[train_size:]

train = train.asfreq('YS')
test = test.asfreq('YS')

## ARDL

In [ ]:
ardl_model_exp = ARDL(train['export'], lags=3, exog=train[['who_com_prices']])
ardl_model_imp = ARDL(train['import'], lags=3, exog=train[['who_com_prices']])

ardl_fit_exp = ardl_model_exp.fit()
ardl_fit_imp = ardl_model_imp.fit()

ardl_forecast_exp = ardl_fit_exp.predict(start=test.index[0], end=test.index[-1], exog_oos=test[['who_com_prices']])
ardl_forecast_imp = ardl_fit_imp.predict(start=test.index[0], end=test.index[-1], exog_oos=test[['who_com_prices']])

errors['ARDL_export'] = (mean_squared_error(test['export'], ardl_forecast_exp), rmse(test['export'], ardl_forecast_exp))
errors['ARDL_import'] = (mean_squared_error(test['import'], ardl_forecast_imp), rmse(test['import'], ardl_forecast_imp))

## Moving Average

In [ ]:
mov_avg_exp = train['export'].rolling(window=2).mean().iloc[-1]
mov_avg_imp = train['import'].rolling(window=2).mean().iloc[-1]

mov_avg_forecast_exp = np.full(len(test), mov_avg_exp)
mov_avg_forecast_imp = np.full(len(test), mov_avg_imp)

errors['MA_export'] = (mean_squared_error(test['export'], mov_avg_forecast_exp), rmse(test['export'], mov_avg_forecast_exp))
errors['MA_import'] = (mean_squared_error(test['import'], mov_avg_forecast_imp), rmse(test['import'], mov_avg_forecast_imp))

## Exponential Smoothing

In [ ]:
exp_smooth_exp = ExponentialSmoothing(train['export'], trend='add', damped_trend=True).fit()
exp_smooth_imp = ExponentialSmoothing(train['import'], trend='add', damped_trend=True).fit()

exp_smooth_forecast_exp = exp_smooth_exp.forecast(len(test))
exp_smooth_forecast_imp = exp_smooth_imp.forecast(len(test))

errors['ES_export'] = (mean_squared_error(test['export'], exp_smooth_forecast_exp), rmse(test['export'], exp_smooth_forecast_exp))
errors['ES_import'] = (mean_squared_error(test['import'], exp_smooth_forecast_imp), rmse(test['import'], exp_smooth_forecast_imp))

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [ ]:
best_export_model = min(errors, key=lambda x: errors[x][0] if 'export' in x else float('inf'))
best_import_model = min(errors, key=lambda x: errors[x][0] if 'import' in x else float('inf'))

print("Model Performance (MSE, RMSE):")
for model, (mse, rmse_val) in errors.items():
    print(f"{model}: MSE={mse:.2f}, RMSE={rmse_val:.2f}")

print(f"\nBest Model for Export: {best_export_model}")
print(f"Best Model for Import: {best_import_model}")

Model Performance (MSE, RMSE):
ARDL_export: MSE=4761517357747899727872.00, RMSE=69003748867.35
ARDL_import: MSE=566399296440284020736.00, RMSE=23799144867.84
MA_export: MSE=6573824090946492432384.00, RMSE=81079122411.05
MA_import: MSE=1934791534747990884352.00, RMSE=43986265296.66
ES_export: MSE=4321095582701641531392.00, RMSE=65735040752.26
ES_import: MSE=530669966056879882240.00, RMSE=23036275003.93

Best Model for Export: ES_export
Best Model for Import: ES_import


## Future Predictions

In [ ]:
future_years = 3

if "ARDL" in best_export_model:
    best_model_exp = ARDL(yearly_trade_data_cleaned['export'], lags=2, exog=yearly_trade_data_cleaned[['who_com_prices']]).fit()
    best_model_exp = best_model_imp.predict(start=yearly_trade_data_cleaned.index[-1] + pd.DateOffset(years = 1), end=yearly_trade_data_cleaned.index[-1] + pd.DateOffset(years = 1 + future_years), exog_oos=yearly_trade_data_cleaned[['who_com_prices']])
elif "MA" in best_export_model:
    best_model_exp = yearly_trade_data_cleaned['export'].rolling(window=2).mean().iloc[-1]
    best_model_exp = np.full(future_years, best_model_exp)
else:
    best_model_exp = ExponentialSmoothing(yearly_trade_data_cleaned['export'], trend='add', damped_trend=True).fit()
    best_model_exp = best_model_exp.forecast(future_years)

if "ARDL" in best_import_model:
    best_model_imp = ARDL(yearly_trade_data_cleaned['import'], lags=2, exog=yearly_trade_data_cleaned[['who_com_prices']]).fit()
    best_model_imp = best_model_imp.predict(start=yearly_trade_data_cleaned.index[-1] + pd.DateOffset(years = 1), end=yearly_trade_data_cleaned.index[-1] + pd.DateOffset(years = future_years), exog_oos=yearly_trade_data_cleaned[['who_com_prices']])
elif "MA" in best_import_model:
    best_model_imp = yearly_trade_data_cleaned['import'].rolling(window=2).mean().iloc[-1]
    best_model_imp = np.full(future_years, best_model_imp)
else:
    best_model_imp = ExponentialSmoothing(yearly_trade_data_cleaned['import'], trend='add', damped_trend=True).fit()
    best_model_imp = best_model_imp.forecast(future_years)


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/holtwinters/model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



In [ ]:
future_dates = pd.date_range(start=yearly_trade_data_cleaned.index[-1] + pd.DateOffset(years=1), periods=future_years, freq='YS')
future_forecast = pd.DataFrame({'export': best_model_exp, 'import': best_model_imp})

print("\nFuture Forecast:")
future_forecast


Future Forecast:


,export,import
2025-01-01,2.768278e+11,2.476822e+11
2026-01-01,2.887959e+11,2.614828e+11
2027-01-01,3.006443e+11,2.751454e+11


In [ ]:
final_data = pd.concat([yearly_trade_data_cleaned, future_forecast])[['export', 'import']]
final_data.index = final_data.index.year
final_data['type'] = ['Actual' if year <= 2024 else 'Predicted' for year in final_data.index]
final_data = final_data.reset_index(names=['year'])
final_data.head()

,year,export,import,type
0,2003,6.105825e+10,3.255069e+10,Actual
1,2004,7.158461e+10,4.652453e+10,Actual
2,2005,8.565995e+10,5.770088e+10,Actual
3,2006,1.007986e+11,6.106547e+10,Actual
4,2007,1.141009e+11,7.447343e+10,Actual


In [ ]:
final_data_melt = final_data.melt(id_vars=['year', 'type'], value_vars=['export', 'import'], var_name='Trade Type', value_name='Trade Value')
fig = px.line(final_data_melt,
              x='year',
              y='Trade Value',
              color='Trade Type',
              line_dash='type',
              labels={'Trade Value': 'Trade Value (USD)', 'year': 'Year'},
              template="plotly_white",
              markers=True)

fig.update_xaxes(tickmode='linear')
fig.show()